# Обработка текстовых данных

## Вступление
Многие практические задачи так или иначе могут вовлекать в себя работу с текстовыми данными, например:

- классификация текстов
    - анализ тональности (например, позитивный/негативный отзыв)
    - фильтрация спама
    - по теме или жанру
- машинный перевод
- распознавание и синтез речи
- извлечение информации
    - именованные сущности (например, извлечение имен, локаций, названий организаций)
    - извлечение фактов и событий
- кластеризация текстов
- оптическое распознавание символов
- проверка правописания
- вопросно-ответные системы, информационный поиск
- суммаризация текстов
- генерация текстов

В целом, алгоритм работы с текстовыми данными можно разбить на такие шаги:

- предобработка сырых данных
- токенизация (создание словаря)
- обработка словаря (удаление стоп-слов и пунктуации)
- обработка токенов (лемматизация / стемминг)
- векторизация текста (bag of words, TF-IDF, etc)

### Структура ноутбука:
1. Токенизация
2. Стоп-слова и пунктуация
3. Лемматизация и стемминг
4. Bag-of-words и TD-IDF
5. Решение задачи с текстовыми данными
6. Регулярные выражения
7. Немножко про категориальные признаки
8. Полный пошаговый пайплайн работы с текстом

**Для работы с тектом я буду в основном пользоваться библиотекой `nltk` (Natural Language Toolkit) — это популярная и очень мощная библиотека Python для обработки и анализа естественного языка (NLP).**

*Что можно реализовать с помощью nltk:*

- разбиение текста на предложения и слова (токенизация)
- удаление стоп-слов
- стемминг (приведение слова к корню)
- лемматизация (приведение к начальной форме)
- определение частей речи
- обработка синтаксиса и деревьев разбора
- работа с большими текстовыми корпусами (коллекциями текстов)
- построение частотных словарей, анализ n-грамм
- подготовка текста для последующего машинного обучения

In [2]:
#загрузка библиотеки nltk
# !pip install nltk

In [3]:
#загрузка библиотеки nltk
# !pip install pandas

## 1. Токенизация

Токенизировать — значит, поделить текст на отдельные смысловые единицы (как правило слова, но не обязательно), или по другому *токены*. Самый тривиальный способ токенизировать текст — разделить на слова по пробелам с помощью `split`. Но `split` упускает очень много всего, например, не отделяет пунктуацию от слов. Кроме этого есть ещё много менее очевидных проблем, возникающих при такой токенизации, поэтому на практике всегда используют готовые токенизаторы.

In [4]:
import warnings

import nltk
import pandas as pd
from nltk.tokenize import word_tokenize

warnings.filterwarnings("ignore")

In [5]:
nltk.download("punkt_tab", quiet=True)

True

In [6]:
example = "Но не каждый хочет что-то исправлять:("

In [7]:
print('Исходный текст:', example)
print('Токенизация c помощью split():', example.split())
print('Токенизация c помощью готового токенизатора:', word_tokenize(example))

Исходный текст: Но не каждый хочет что-то исправлять:(
Токенизация c помощью split(): ['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять:(']
Токенизация c помощью готового токенизатора: ['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':', '(']


В `nltk` есть много разных токенизаторов, посмотреть на них можно так:

In [8]:
from nltk import tokenize

print('Всего встроенных в nltk токенизаторов:', len(dir(tokenize)))
#первые 10
dir(tokenize)[:10]

Всего встроенных в nltk токенизаторов: 61


['BlanklineTokenizer',
 'LegalitySyllableTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'NLTKWordTokenizer',
 'PunktSentenceTokenizer',
 'PunktTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer']

Можно получить индексы начала и конца каждого токена:

In [9]:
wh_tok = tokenize.WhitespaceTokenizer()
list(wh_tok.span_tokenize(example))

[(0, 2), (3, 5), (6, 12), (13, 18), (19, 25), (26, 38)]

Некторые токенизаторы ведут себя специфично:

In [10]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

['do', "n't", 'stop', 'me']

Для некоторых задач это может быть полезно.

А некоторые предназначены вообще не для текста на естественном языке:

In [11]:
tokenize.SExprTokenizer().tokenize("(a (b c)) d e (f)")

['(a (b c))', 'd', 'e', '(f)']

Есть токенизатор, который может быть полезен для работы с твитами или сообщениями из соц. сетей. Он сохранит смайлики, хештеги и т.п.

In [12]:
from nltk.tokenize import TweetTokenizer

tw = TweetTokenizer()
tw.tokenize(example)

['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':(']

## 3. Стоп-слова и пунктуация

*Стоп-слова* — это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [13]:
nltk.download("stopwords", quiet=True)

True

In [14]:
from nltk.corpus import stopwords

print(stopwords.words("russian"))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [15]:
print(stopwords.words("english"))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [16]:
#список русских стоп слов
rus_stop_words_list = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                        'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 
                        'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже',
                        'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом',
                        'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их',
                        'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда',  
                        'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 
                        'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об',
                        'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 
                        'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя',
                        'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']
#список английских стоп слов
eng_stop_words_list = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as',
                        'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd',
                        'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 
                        'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll",
                        'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 
                        'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more',
                        'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only',
                        'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she'd", "she'll", 
                        "she's", 'should', 'shouldn', "shouldn't", "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 
                        'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', "they'd", "they'll", "they're", "they've", 'this', 'those', 
                        'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', "we'd", "we'll", "we're", 'were', 
                        'weren', "weren't", "we've", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 
                        'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", 'your', "you're", 'yours', 'yourself', 'yourselves', "you've"]

In [17]:
from string import punctuation

#знаки пунктуации
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
#стоп слова + знаки пунктуации
noise = stopwords.words("russian") + list(punctuation)

## 3. Лемматизация и стемминг
### 3.1 Лемматизация

[**Лемматизация**](https://en.wikipedia.org/wiki/Lemmatisation) — процесс приведения слова к его нормальной форме (**лемме**):
- для существительных — именительный падеж, единственное число;
- для прилагательных — именительный падеж, единственное число, мужской род;
- для глаголов, причастий, деепричастий — глагол в инфинитиве.

Например, токены «пью», «пил», «пьет» перейдут в «пить». Почему это хорошо?
* Во-первых, мы хотим рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лемматизации выкидываем мы только её.

Для русского есть два хороших лемматизатора: `mystem` и `pymorphy`.

#### [Mystem](https://tech.yandex.ru/mystem/)

Mystem — это пример популярной библиотеки для лемматизации. Как с ним работать:
* скачать mystem и запускать [из терминала с разными параметрами](https://tech.yandex.ru/mystem/doc/)
* использовать обёртку для питона [pymystem3](https://pythonhosted.org/pymystem3/pymystem3.html) (она медленнее, но удобнее в использовании)

In [19]:
# !pip install pymystem3

In [20]:
from pymystem3 import Mystem

mystem_analyzer = Mystem()

Мы инициализировали Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin — путь к `mystem`, если их несколько
* grammar_info — нужна ли грамматическая информация или только леммы (по умолчанию нужна)
* disambiguation — нужно ли снятие [омонимии](https://ru.wikipedia.org/wiki/%D0%9E%D0%BC%D0%BE%D0%BD%D0%B8%D0%BC%D1%8B) - дизамбигуация (по умолчанию нужна)
* entire_input — нужно ли сохранять в выводе все (пробелы, например), или можно выкинуть (по умолчанию оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

Можно просто лемматизировать текст:

In [21]:
print(mystem_analyzer.lemmatize(example))

['но', ' ', 'не', ' ', 'каждый', ' ', 'хотеть', ' ', 'что-то', ' ', 'исправлять', ':(\n']


### [Pymorphy](http://pymorphy3.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

In [22]:
# !pip install pymorphy3

In [23]:
from pymorphy3 import MorphAnalyzer

pymorphy3_analyzer = MorphAnalyzer()

pymorphy3 работает с отдельными словами. Если дать ему на вход предложение, то он его просто не лемматизирует, т.к. не понимает.

Метод MorphAnalyzer.parse() принимает слово и возвращает все возможные его разборы.

У каждого разбора есть тег. Тег — это набор граммем, характеризующих данное слово. Например, тег 'VERB,perf,intr plur,past,indc' означает, что слово — глагол (VERB) совершенного вида (perf), непереходный (intr), множественного числа (plur), прошедшего времени (past), изъявительного наклонения (indc).


In [24]:
ana = pymorphy3_analyzer.parse("хочет")
ana

[Parse(word='хочет', tag=OpencorporaTag('VERB,impf,tran sing,3per,pres,indc'), normal_form='хотеть', score=1.0, methods_stack=((DictionaryAnalyzer(), 'хочет', 3136, 5),))]

In [25]:
ana[0].normal_form

'хотеть'

### mystem vs. pymorphy

1) *mystem работает невероятно медленно под windows на больших текстах

3) *Снятие омонимии*. Mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy3 берет на вход одно слово и соответственно вообще не умеет дизамбигуировать по контексту.

In [26]:
homonym1 = "За время обучения я прослушал больше сорока курсов."
homonym2 = "Сорока своровала блестящее украшение со стола."

In [27]:
# корректно определил части речи
# NUM — числительное
# S — существительное
print(mystem_analyzer.analyze(homonym1)[-5])
print(mystem_analyzer.analyze(homonym2)[0])

{'analysis': [{'lex': 'сорок', 'wt': 0.8710292664, 'gr': 'NUM=(пр|дат|род|твор)'}], 'text': 'сорока'}
{'analysis': [{'lex': 'сорока', 'wt': 0.1210970041, 'gr': 'S,жен,од=им,ед'}], 'text': 'Сорока'}


**mystem:**

In [28]:
mystem_analyzer.analyze(homonym1)

[{'analysis': [{'lex': 'за', 'wt': 1, 'gr': 'PR='}], 'text': 'За'},
 {'text': ' '},
 {'analysis': [{'lex': 'время', 'wt': 1, 'gr': 'S,сред,неод=(вин,ед|им,ед)'}],
  'text': 'время'},
 {'text': ' '},
 {'analysis': [{'lex': 'обучение',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)'}],
  'text': 'обучения'},
 {'text': ' '},
 {'analysis': [{'lex': 'я', 'wt': 0.9999716281, 'gr': 'SPRO,ед,1-л=им'}],
  'text': 'я'},
 {'text': ' '},
 {'analysis': [{'lex': 'прослушивать',
    'wt': 1,
    'gr': 'V,пе=прош,ед,изъяв,муж,сов'}],
  'text': 'прослушал'},
 {'text': ' '},
 {'analysis': [{'lex': 'много', 'wt': 0.0002164204767, 'gr': 'ADV=срав'}],
  'text': 'больше'},
 {'text': ' '},
 {'analysis': [{'lex': 'сорок',
    'wt': 0.8710292664,
    'gr': 'NUM=(пр|дат|род|твор)'}],
  'text': 'сорока'},
 {'text': ' '},
 {'analysis': [{'lex': 'курс', 'wt': 0.6284122441, 'gr': 'S,муж,неод=род,мн'}],
  'text': 'курсов'},
 {'text': '.'},
 {'text': '\n'}]

In [29]:
mystem_analyzer.analyze(homonym2)

[{'analysis': [{'lex': 'сорока', 'wt': 0.1210970041, 'gr': 'S,жен,од=им,ед'}],
  'text': 'Сорока'},
 {'text': ' '},
 {'analysis': [{'lex': 'своровать',
    'wt': 1,
    'gr': 'V,сов,пе=прош,ед,изъяв,жен'}],
  'text': 'своровала'},
 {'text': ' '},
 {'analysis': [{'lex': 'блестящий',
    'wt': 0.6831493248,
    'gr': 'A=(вин,ед,полн,сред|им,ед,полн,сред|срав)'}],
  'text': 'блестящее'},
 {'text': ' '},
 {'analysis': [{'lex': 'украшение',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,ед|им,ед)'}],
  'text': 'украшение'},
 {'text': ' '},
 {'analysis': [{'lex': 'со', 'wt': 1, 'gr': 'PR='}], 'text': 'со'},
 {'text': ' '},
 {'analysis': [{'lex': 'стол', 'wt': 1, 'gr': 'S,муж,неод=род,ед'}],
  'text': 'стола'},
 {'text': '.'},
 {'text': '\n'}]

**pymorphy:**

In [30]:
for word in homonym1.split():
    print(pymorphy3_analyzer.parse(word))

[Parse(word='за', tag=OpencorporaTag('PREP'), normal_form='за', score=1.0, methods_stack=((DictionaryAnalyzer(), 'за', 24, 0),))]
[Parse(word='время', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='время', score=0.861936, methods_stack=((DictionaryAnalyzer(), 'время', 563, 3),)), Parse(word='время', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='время', score=0.138063, methods_stack=((DictionaryAnalyzer(), 'время', 563, 0),))]
[Parse(word='обучения', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='обучение', score=0.968253, methods_stack=((DictionaryAnalyzer(), 'обучения', 77, 2),)), Parse(word='обучения', tag=OpencorporaTag('NOUN,inan,neut plur,nomn'), normal_form='обучение', score=0.015873, methods_stack=((DictionaryAnalyzer(), 'обучения', 77, 13),)), Parse(word='обучения', tag=OpencorporaTag('NOUN,inan,neut plur,accs'), normal_form='обучение', score=0.015873, methods_stack=((DictionaryAnalyzer(), 'обучения', 77, 18),))]
[Parse(word='я', tag=O

In [31]:
for word in homonym2.split():
    print(pymorphy3_analyzer.parse(word))

[Parse(word='сорока', tag=OpencorporaTag('NUMR gent'), normal_form='сорок', score=0.68, methods_stack=((DictionaryAnalyzer(), 'сорока', 2920, 1),)), Parse(word='сорока', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='сорока', score=0.08, methods_stack=((DictionaryAnalyzer(), 'сорока', 421, 0),)), Parse(word='сорока', tag=OpencorporaTag('NUMR ablt'), normal_form='сорок', score=0.08, methods_stack=((DictionaryAnalyzer(), 'сорока', 2920, 4),)), Parse(word='сорока', tag=OpencorporaTag('NUMR loct'), normal_form='сорок', score=0.08, methods_stack=((DictionaryAnalyzer(), 'сорока', 2920, 5),)), Parse(word='сорока', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='сорока', score=0.04, methods_stack=((DictionaryAnalyzer(), 'сорока', 44, 0),)), Parse(word='сорока', tag=OpencorporaTag('NUMR datv'), normal_form='сорок', score=0.04, methods_stack=((DictionaryAnalyzer(), 'сорока', 2920, 2),))]
[Parse(word='своровала', tag=OpencorporaTag('VERB,perf,tran femn,sing,past,indc'),

### 3.2 Стемминг

В отличие от лемматизации, при применении стемминга у всех слов отбрасываются аффиксы (окончания и суффиксы), что необязательно приводит слова к формам, существующим в рассматриваемом языке. [**Snowball**](http://snowball.tartarus.org/) — фрэймворк для написания алгоритмов стемминга. Алгоритмы стемминга отличаются для разных языков и используют знания о конкретном языке: списки окончаний для разных частей речи, разных склонений и т.д.

In [32]:
from nltk.stem.snowball import SnowballStemmer

In [33]:
tokenized_example = word_tokenize(example)

In [34]:
stemmer = SnowballStemmer("russian")
stemmed_example = [stemmer.stem(w) for w in tokenized_example]
print('Исходный текст:', example)
print('Стемминг:', " ".join(stemmed_example))

Исходный текст: Но не каждый хочет что-то исправлять:(
Стемминг: но не кажд хочет что-т исправля : (


Для английского получится что-то такое.

In [35]:
text = 'In my younger and more vulnerable years my father gave me some advice that I\'ve been turning over in my mind ever since.\n"Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven\'t had the advantages that you\'ve had."'
print('Исходный текст:', text)
text_tokenized = [w for w in word_tokenize(text) if w.isalpha()]

print('Токенезация:', text_tokenized)

Исходный текст: In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since.
"Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven't had the advantages that you've had."
Токенезация: ['In', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'I', 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since', 'Whenever', 'you', 'feel', 'like', 'criticizing', 'any', 'one', 'he', 'told', 'me', 'just', 'remember', 'that', 'all', 'the', 'people', 'in', 'this', 'world', 'have', 'had', 'the', 'advantages', 'that', 'you', 'had']


In [36]:
stemmer = SnowballStemmer("english")
text_stemmed = [stemmer.stem(w) for w in text_tokenized]
print('Стемминг:', " ".join(text_stemmed))

Стемминг: in my younger and more vulner year my father gave me some advic that i been turn over in my mind ever sinc whenev you feel like critic ani one he told me just rememb that all the peopl in this world have had the advantag that you had


## 4. Bag-of-words и TF-IDF

Но как же все-таки работать с текстами, используя стандартные методы машинного обучения? Ведь нам нужны объекты выборки, которые описываются числами, а не словами. Иначе говоря, нам нужно *векторизовать* текстовые данные.

### 4.1 Bag-of-words

Пусть у нас имеется коллекция текстов $D = \{d_i\}_{i=1}^l$ (всего $l$ текстов) и словарь всех слов, встречающихся в выборке $V = \{v_j\}_{j=1}^d$ (всего $d$ слов). В этом случае некоторый текст $d_i$ описывается вектором $(x_{ij})_{j=1}^d,$ где
$$x_{ij} = \sum_{v \in d_i} [v = v_j].$$

Таким образом, текст $d_i$ описывается вектором количества вхождений каждого слова из словаря в данный текст.

In [37]:
texts = [
    "I like my cat.",
    "My cat is the most perfect cat.",
    "is this cat or is this bread?",
]

In [38]:
texts_tokenized = [
    " ".join([w for w in word_tokenize(t) if w.isalpha()]) for t in texts
]
texts_tokenized

['I like my cat',
 'My cat is the most perfect cat',
 'is this cat or is this bread']

In [39]:
# !pip install scikit-learn

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vec = CountVectorizer()
X = cnt_vec.fit_transform(texts_tokenized)

In [41]:
cnt_vec.vocabulary_.keys()

dict_keys(['like', 'my', 'cat', 'is', 'the', 'most', 'perfect', 'this', 'or', 'bread'])

In [42]:
X

<3x10 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [43]:
X.toarray()

array([[0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 2, 1, 0, 1, 1, 0, 1, 1, 0],
       [1, 1, 2, 0, 0, 0, 1, 0, 0, 2]])

In [44]:
pd.DataFrame(X.toarray(), columns=cnt_vec.get_feature_names_out())

,bread,cat,is,like,most,my,or,perfect,the,this
0,0,1,0,1,0,1,0,0,0,0
1,0,2,1,0,1,1,0,1,1,0
2,1,1,2,0,0,0,1,0,0,2


Как мы видим у нас по какой-то причине пропало слово `I`. Это связано с тем, что `CountVectorize` для оперделения токенов использует регулярное выражение ```token_pattern=r'(?u)\b\w\w+\b'```, где 
- `\b` — граница слова.
- `\w\w+` — слово из двух или более букв/цифр.

Именно поэтому все слова длиной 1 игнорируются — они не подходят под этот шаблон.

Попробуем исправить парметр `token_pattern=r'(?u)\b\w\w+\b'` на `token_pattern=r'(?u)\b\w+\b'`, чтобы слова длинной из 1 символа так же векторизировались.

In [54]:
cnt_vec_2 = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X_2 = cnt_vec_2.fit_transform(texts_tokenized)

pd.DataFrame(X_2.toarray(), columns=cnt_vec_2.get_feature_names_out())

,bread,cat,i,is,like,most,my,or,perfect,the,this
0,0,1,1,0,1,0,1,0,0,0,0
1,0,2,0,1,0,1,1,0,1,1,0
2,1,1,0,2,0,0,0,1,0,0,2


**Вывод:**
Обычно функцию векторизации задают самостоятельно, чтобы была возможность контролировать разбиение текста на токены, удаление стоп слов и параметры векторизации.

### 4.2 TF-IDF

Заметим, что если слово часто встречается в одном тексте, но почти не встречается в других, то оно получает для данного текста большой вес, ровно так же, как и слова, которые часто встречаются в каждом тексте. Для того чтобы разделять эти такие слова, можно использовать статистическую меру TF-IDF, характеризующую важность слова для конкретного текста. 

---

#### 1. Term Frequency (TF) — "Частота слова в тексте"
Для каждого слова из текста $d$ рассчитаем относительную частоту встречаемости в нем (Term Frequency):
$$
\text{TF}(t, d) = \frac{C(t | d)}{\sum\limits_{k \in d}C(k | d)},
$$
где 
- $C(t | d)$ — сколько раз слово $t$ встречается в тексте $d$. 
- $\sum\limits_{k \in d}C(k | d)$ — сколько **всего** слов в тексте $d$ (с учётом повторяющихся). 

TF показывает, $\underline{насколько}$ $\underline{часто}$ слово встречается в данном тексте.  

**Пример:**  В тексте из 100 слов слово "машина" встречается 4 раза: - TF("машина", d) = 4 / 100 = 0.04 (4% от всех слов)

---

#### 2. Inverse Document Frequency (IDF) — "Обратная частота в других текстах"
Также для каждого слова из текста $d$ рассчитаем обратную частоту встречаемости в корпусе текстов $D$ (Inverse Document Frequency):
$$
\text{IDF}(t, D) = \log\left(\frac{|D|}{|\{d_i \in D \mid t \in d_i\}|}\right)
$$
где
- $|D|$ — общее количество текстов (документов) в коллекции. 
- $|\{d_i \in D \mid t \in d_i\}|$ — сколько текстов содержат слово $t$.

Логарифмирование здесь проводится с целью уменьшить масштаб весов, ибо зачастую в корпусах присутствует очень много текстов.

IDF показывает, $\underline{насколько}$ $\underline{редкое}$ это слово среди всех текстов.  
- Если слово встречается почти во всех текстах (например, "и"), **IDF будет маленьким**. 
- Если слово встречается только в паре статей — **IDF будет большим**.  

**Пример:**  Предположим, у нас 1000 статей ($|D|$ = 1000),  и слово "нейросеть" есть только в 10 из них: - IDF("нейросеть", D) = log(1000 / 10) = log(100) = 2 (если берём десятичный логарифм)

---

#### 3. Общая формула

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

- Если слово часто встречается в **конкретном тексте** (большой TF) и при этом редко встречается в **других текстах** (большой IDF), то **его вес будет большим**. 
- Если слово встречается везде — его вес будет маленьким. 
  
**Пример:**  
- "машина" встречается часто в тексте и редко вообще — TF-IDF будет высоким и слово считается важным для этого текста. 
- "и" встречается часто в тексте, но и вообще везде — TF-IDF маленький, оно считается неважным.


**Есть и другие подходы к определению [TF и IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Definition).**

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(texts_tokenized)

In [56]:
tfidf_vec.vocabulary_.keys()

dict_keys(['like', 'my', 'cat', 'is', 'the', 'most', 'perfect', 'this', 'or', 'bread'])

In [58]:
X

<3x10 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [59]:
X.toarray()

array([[0.        , 0.42544054, 0.        , 0.72033345, 0.        ,
        0.54783215, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.50130994, 0.32276391, 0.        , 0.42439575,
        0.32276391, 0.        , 0.42439575, 0.42439575, 0.        ],
       [0.33976626, 0.20067143, 0.516802  , 0.        , 0.        ,
        0.        , 0.33976626, 0.        , 0.        , 0.67953252]])

In [60]:
pd.DataFrame(X.toarray(), columns=tfidf_vec.get_feature_names_out())

,bread,cat,is,like,most,my,or,perfect,the,this
0,0.000000,0.425441,0.000000,0.720333,0.000000,0.547832,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.501310,0.322764,0.000000,0.424396,0.322764,0.000000,0.424396,0.424396,0.000000
2,0.339766,0.200671,0.516802,0.000000,0.000000,0.000000,0.339766,0.000000,0.000000,0.679533


In [61]:
tfidf_vec_2 = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X_2 = tfidf_vec_2.fit_transform(texts_tokenized)

pd.DataFrame(X_2.toarray(), columns=tfidf_vec_2.get_feature_names_out())

,bread,cat,i,is,like,most,my,or,perfect,the,this
0,0.000000,0.345205,0.584483,0.000000,0.584483,0.000000,0.444514,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.501310,0.000000,0.322764,0.000000,0.424396,0.322764,0.000000,0.424396,0.424396,0.000000
2,0.339766,0.200671,0.000000,0.516802,0.000000,0.000000,0.000000,0.339766,0.000000,0.000000,0.679533


## 5. Решение задачи с текстовыми данными

Будем решать задачу классификации твитов по тональности. Возьмём датасет из твитов, в котором про каждый твит известно, как он эмоционально окрашен: положительно или отрицательно. Задача: предсказывать эмоциональную окраску. Классификацию по тональности используют, например, в рекомендательных системах, чтобы понять, понравилось ли людям кафе, кино, etc.

Скачиваем куски датасета ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [ ]:
# !wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
# !wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2025-08-25 23:02:49--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Распознаётся www.dropbox.com (www.dropbox.com)… 2620:100:6022:18::a27d:4212, 162.125.70.18
Подключение к www.dropbox.com (www.dropbox.com)|2620:100:6022:18::a27d:4212|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://www.dropbox.com/scl/fi/6mg7rw3wltux83q2o4ah4/positive.csv?rlkey=cvruhzofza9kkfxwzyp2vskfd [переход]
--2025-08-25 23:02:50--  https://www.dropbox.com/scl/fi/6mg7rw3wltux83q2o4ah4/positive.csv?rlkey=cvruhzofza9kkfxwzyp2vskfd
Повторное использование соединения с [www.dropbox.com]:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://ucd413931a677aa771e6d2f1f607.dl.dropboxusercontent.com/cd/0/inline/CwGw-WgWVF8haRjTbwz3HMpTGXXTWtwpVziWY5skUFgKmmAaHkQ_Z44orMW1OfwZ4w-mFBLcJPsxUUPy7B1xg0HIQziJJ7nrzIUr10f26TFvm_UDjFC4UO0F0YV_7oJXawc/file# [переход]
--2025-08-25 23:02:50--  https://ucd413931a677aa771e6d2f1f607.dl.dropboxusercontent.com/cd/0/

In [63]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

In [64]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv("positive.csv", sep=";", usecols=[3], names=["text"])
positive["label"] = "positive"
negative = pd.read_csv("negative.csv", sep=";", usecols=[3], names=["text"])
negative["label"] = "negative"
df = pd.concat([positive, negative])

In [66]:
display(df.head(), df.tail())

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [67]:
df.shape

(226834, 2)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226834 entries, 0 to 111922
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 5.2+ MB


In [70]:
#Разобьем исходный датасет на train и test
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label, random_state=13)

#### n-граммы

n-граммы — это последовательности n токенов из исходного текста. В простейшем случае это могут быть последовательности из букв или последовательности из слов. Давайте посмотрим подробнее на примере.

In [71]:
from nltk import ngrams

In [72]:
sent = "Если б мне платили каждый раз".split()
list(ngrams(sent, 1))  # униграммы

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

In [73]:
list(ngrams(sent, 2))  # биграммы

[('Если', 'б'),
 ('б', 'мне'),
 ('мне', 'платили'),
 ('платили', 'каждый'),
 ('каждый', 'раз')]

In [74]:
list(ngrams(sent, 3))  # триграммы

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

In [75]:
list(ngrams(sent, 5))  # ... пентаграммы?

[('Если', 'б', 'мне', 'платили', 'каждый'),
 ('б', 'мне', 'платили', 'каждый', 'раз')]

В качестве альтернативы можно пользоваться `CountVectorizer`, который работает так:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности количества токенов в нашем словаре
* заполняет каждый i-тый элемент количеством вхождений токена в данный документ

Параметр `ngram_range` отвечает за то, какие n-граммы мы используем в качестве фичей:
- `ngram_range=(1, 1)` — униграммы
- `ngram_range=(3, 3)` — триграммы
- `ngram_range=(1, 3)` — униграммы, биграммы и триграммы

### 5.1 Обучение моделей

Обучим наш первый бейзлайн — логрег на униграммах!

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)  # bow — bag of words (мешок слов)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.76      0.77      0.76     27957
    positive       0.77      0.76      0.77     28752

    accuracy                           0.77     56709
   macro avg       0.76      0.77      0.76     56709
weighted avg       0.77      0.77      0.77     56709



Попробуем сделать то же самое для биграмм и триграмм.

In [78]:
vec2 = CountVectorizer(ngram_range=(2, 2))
bow2 = vec2.fit_transform(x_train)  # bow — bag of words (мешок слов)
bow_test2 = vec2.transform(x_test)

scaler2 = MaxAbsScaler()
bow2 = scaler2.fit_transform(bow2)
bow_test2 = scaler2.transform(bow_test2)

clf2 = LogisticRegression(max_iter=500, random_state=42)
clf2.fit(bow2, y_train)
pred2 = clf2.predict(bow_test2)
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

    negative       0.70      0.74      0.72     27957
    positive       0.73      0.69      0.71     28752

    accuracy                           0.71     56709
   macro avg       0.71      0.71      0.71     56709
weighted avg       0.71      0.71      0.71     56709



In [79]:
vec3 = CountVectorizer(ngram_range=(3, 3))
bow3 = vec3.fit_transform(x_train)  # bow — bag of words (мешок слов)
bow_test3 = vec3.transform(x_test)

scaler3 = MaxAbsScaler()
bow3 = scaler3.fit_transform(bow3)
bow_test3 = scaler3.transform(bow_test3)

clf3 = LogisticRegression(max_iter=500, random_state=42)
clf3.fit(bow3, y_train)
pred3 = clf3.predict(bow_test3)
print(classification_report(y_test, pred3))

              precision    recall  f1-score   support

    negative       0.72      0.47      0.57     27957
    positive       0.61      0.82      0.70     28752

    accuracy                           0.65     56709
   macro avg       0.67      0.65      0.64     56709
weighted avg       0.67      0.65      0.64     56709



Как видно биграммы и триграммы дают худшую точность относительно униграмм.

А теперь повторим процедуру для TF-IDF (на униграммах).

In [81]:
vec = TfidfVectorizer(ngram_range=(1, 1))
vec_train = vec.fit_transform(x_train)
vec_test = vec.transform(x_test)

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(vec_train, y_train)
pred_tfidf = clf.predict(vec_test)
print(classification_report(y_test, pred_tfidf))

              precision    recall  f1-score   support

    negative       0.77      0.75      0.76     27957
    positive       0.76      0.78      0.77     28752

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



### 5.2 О важности эксплоративного анализа

Иногда пунктуация бывает и не шумом. Главное — отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [83]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.95      0.97      0.96     27957
    positive       0.97      0.95      0.96     28752

    accuracy                           0.96     56709
   macro avg       0.96      0.96      0.96     56709
weighted avg       0.96      0.96      0.96     56709



Как мы видим отказ от очистки от пунктуации (в базовом случае, используется внутренний токенизатор, который как мы помним работает таким паттерном `token_pattern=r'(?u)\b\w\w+\b'`, при указании параметра `tokenizer=word_tokenize` мы ссылаемся на внешний токенизатор и внутренний токенизатор уже не вмешивается в работу), привел к внезапному росту всех метрик, значит в данном случаи какие-то пунктуационные символы являются очень значимыми. Посмотрим на наиболее значимый из них.

In [84]:
vec.get_feature_names_out()[np.argmax(clf.coef_)]

')'

Можно построить предскозание основываясь на наличии только одного этого токена и посмотреть на точность.

In [87]:
cool_token = vec.get_feature_names_out()[np.argmax(clf.coef_)]
pred = ["positive" if cool_token in tweet else "negative" for tweet in x_test]
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.85      1.00      0.92     27957
    positive       1.00      0.83      0.91     28752

    accuracy                           0.91     56709
   macro avg       0.93      0.92      0.91     56709
weighted avg       0.93      0.91      0.91     56709



In [88]:
cool_token = vec.get_feature_names_out()[np.argmax(clf.coef_)]
tweets_with_cool_token = [tweet for tweet in x_train if cool_token in tweet]
np.random.seed(42)
for tweet in np.random.choice(tweets_with_cool_token, size=10, replace=False):
    print(tweet)

“@Fashionbar_uz: Ladies Monday . Каждый понедельник для всех девушек кальяны от заведения. #fashionbar http://t.co/ZlvxOcZPR4” zaviduyu))
Гримерка-лук))) #evabristol #performance #gig #vocaldiva #гастроли #гитис #театр http://t.co/S60OvyyTS6
RT @alivfedorov: http://t.co/DvYLJaPHxR Девушки это самые хитрые создания! так что даже не пытайся их обмануть!:)
@pavelsheremet @ukrpravda_news @varlamov хотя исторически правильно желто-синий:)
Понятия не имею чем меня привлекла эта картинка!)))http://t.co/twqP8zyh1A
@Sveta12126 ну или пусть Лиама пришлет ко мне) своего младшенького. Как сказала Кэтрин: "Это нормально - любить двоих" ахах
Вышел в свет новый каталог запасных частей и деталей ТМК! Звоните - подарим :)
@u_alekseeva_17 видужуй:3
Я там тепер буду кожну середу)
29-й выпуск Дроидкаста будет не против ваших плюсов на Хабре   ;)
@nemoniga а я вот знаю :) нам на политической географии рассказывал душечка Гурин


Как мы увидели модель классификации, которая делает предсказание только исходя из факта наличичя символа `)` в тексте сравнима по точности с нашей самой точной моделью линейной регрессии. 

**Вывод:** Для данного набора данных, наличиче в тексте символа `)` - является значимым фактором, чтобы сделать предсказание в пользу класса `positive`

### 5.3 Символьные n-граммы

Теперь в качестве признаков используем, например, униграммы символов:

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), analyzer="char") #оставляем только едиинчные символы в качестве токенов
bow = vec.fit_transform(x_train)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=500, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.99      0.97      0.98     27957
    positive       0.98      0.99      0.98     28752

    accuracy                           0.98     56709
   macro avg       0.98      0.98      0.98     56709
weighted avg       0.98      0.98      0.98     56709



В общем-то, теперь уже понятно, почему на этих данных здесь 1. Так или иначе, на символах классифицировать тоже можно: для некоторых задач (например, для определения языка) признаки-символьные n-граммы могут внести серьезный вклад в качество модели. Ещё одна замечательная особенность признаков-символов: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## 6. Регулярные выражения

Часто бывает так, что для конкретного случая нужен особый способ токенизации, и надо самостоятельно написать правило шаблонного типа для определения того, что такое токен. Или, например, перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее. В таких задачах могут помочь регулярные выражения. Навык полезный, давайте в нём тоже потренируемся.

In [91]:
import re

### findall
возвращает список всех найденных совпадений

- `?` : ноль или одно повторение
- `*` : ноль или более повторений
- `+` : одно или более повторений
- `.`: любой символ

In [92]:
result = re.findall("ab+c.", "abcdefghijkabcabcxabc")
print(result)

['abcd', 'abca']


In [93]:
re.findall("ab+c.", "abbbca")

['abbbca']

### split
разделяет строку по заданному шаблону


In [94]:
result = re.split(",", "itsy, bitsy, teenie, weenie")
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [95]:
result = re.split(",", "itsy, bitsy, teenie, weenie", maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [96]:
result = re.sub("a", "b", "abcabc")
print(result)

bbcbbc


При этом в качестве repl, можно передавать не только строку, но и функцию, которая принимает на вход [Match](https://docs.python.org/3/library/re.html#match-objects) объект. Можно делать что-то типо этого:

In [97]:
counter = 0


def count(match):
    global counter
    counter += 1
    return f"(a#{counter})"


re.sub("a", count, "abcabc")

'(a#1)bc(a#2)bc'

Кстати, c объектами типа re.Match работают и многие другие методы re. Например, метод re.finditer в отличии от re.findall будет возвращать те самые re.Match.

In [98]:
for match in re.finditer("ab+c.", "abcdefghijkabcabcxabc"):
    print(match)

<re.Match object; span=(0, 4), match='abcd'>
<re.Match object; span=(11, 15), match='abca'>


Помимо найденных строчек объекты Match также, например, содержат информацию о позиции найденного "совпадения" в строке (span)

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile("[А-Яа-яё\-]+")
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

In [ ]:
#Так можно вернуть списко всех доменов из строки с разынми почтами
emails = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
domains = re.compile(r'@[\w.-]+')
domains.findall(emails)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']

## 7. Немножко про категориальные признаки

Мы уже говорили, что кодировать категориальные признаки просто в виде чисел — не очень хорошая идея. Это задаёт некоторый порядок, которого на категориальных переменных может и не быть. Существует три основных способа обработки категориальных значений:
- Label encoding
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке

Основная идея счетчиков заключается в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории. Формально это можно записать так:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

Сравним два последних метода на нашем любимом Титанике, но на этот раз не будем сильно заморачиваться с обработкой пропусков.

In [102]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# data = pd.read_csv(
#     "https://raw.githubusercontent.com/iad34/seminars/master/materials/data_sem1.csv",
#     sep=";",
# )

# data["Age"] = data["Age"].fillna(data.groupby("Pclass")["Age"].transform("median"))
# data.drop("Cabin", axis=1, inplace=True)
# data.drop("Name", axis=1, inplace=True)
# data.drop("Ticket", axis=1, inplace=True)
# data.dropna(inplace=True)
# data.to_csv('cat_feat_enc_exanple.csv')

data = pd.read_csv('cat_feat_enc_exanple.csv')

### 7.1 OHE

In [106]:
data_ohe = pd.get_dummies(data, drop_first=True)

y = data_ohe["Survived"]
data_ohe = data_ohe.drop(["Survived"], axis=1)
data_ohe.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_male,Sex_unknown,Embarked_Q,Embarked_S
0,1,3,22.0,1,0,7.2500,True,False,False,True
1,2,1,38.0,1,0,71.2833,False,False,False,False
2,3,3,26.0,0,0,7.9250,False,False,False,True
3,4,1,35.0,1,0,53.1000,False,False,False,True
4,5,3,35.0,0,0,8.0500,True,False,False,True


In [107]:
X_train, X_test, y_train, y_test = train_test_split(
    data_ohe, y, test_size=0.2, random_state=42, shuffle=True
)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
print(f"ROC AUC: {roc_auc_score(prediction, y_test):.2f}")

ROC AUC: 0.77


### 7.2 Mean target encoding

In [108]:
X_train, X_test, y_train, y_test = train_test_split(
    data, y, test_size=0.2, random_state=42, shuffle=True
)

In [109]:
mean_target = X_train.groupby("Sex")["Survived"].mean()
X_train.loc[:, "Sex"] = X_train["Sex"].replace(mean_target)
X_test.loc[:, "Sex"] = X_test["Sex"].replace(mean_target)

mean_target_e = X_train.groupby("Embarked")["Survived"].mean()
X_train.loc[:, "Embarked"] = X_train["Embarked"].replace(mean_target_e)
X_test.loc[:, "Embarked"] = X_test["Embarked"].replace(mean_target_e)

X_train.drop(["Survived"], axis=1, inplace=True)
X_test.drop(["Survived"], axis=1, inplace=True)

In [110]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
print(f"ROC AUC: {roc_auc_score(prediction, y_test):.2f}")

ROC AUC: 0.79


Кодирование признаков с помощью счетчиков приведённом выше может приводить к переобучению.

Чтобы бороться с этим, можно экспериментировать с разными модификациями:

1. Вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
2. Вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
3. Добавлять шум в посчитанные признаки